# Importing the libraries

In [ ]:
!nvidia-smi

Sat Dec 26 11:53:47 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.27.04    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Importing the training set

In [ ]:
dataset_train = pd.read_csv('Train.csv')
training_set = dataset_train.iloc[:, 1:].values

# Feature Scaling

In [ ]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0,1))
training_set_scaled = sc.fit_transform(training_set)

# Creating a data structure with 870 timesteps and 1 output

In [ ]:
X_train = []
y_train = []
for i in range(1170, len(training_set)):
  X_train.append(training_set_scaled[i-1170:i,:])
  y_train.append(training_set_scaled[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)

# Reshaping

In [ ]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 5))

# Importing the keras libraries and packages

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

# Initializing the RNN

In [ ]:
regressor = Sequential()

#Adding the first LSTM layer and some Dropout regularisation

In [ ]:
regressor.add(LSTM(units = 128, return_sequences = True, input_shape = (X_train.shape[1], 5)))
regressor.add(Dropout(0.2))

#Adding the second LSTM layer and some Dropout regularisation

In [ ]:
regressor.add(LSTM(units = 128, return_sequences = True))
regressor.add(Dropout(0.2))

# Adding the third LSTM layer and some Dropout regularisation

In [ ]:
regressor.add(LSTM(units = 128, return_sequences = True))
regressor.add(Dropout(0.2))

#Adding the fourth LSTM layer and some Dropout regularisation

In [ ]:
regressor.add(LSTM(units = 128, return_sequences = True))
regressor.add(Dropout(0.2))

#Adding the fifth LSTM layer and some Dropout regularisation

In [ ]:
regressor.add(LSTM(units = 128))
regressor.add(Dropout(0.2))

#The output layer

In [ ]:
regressor.add(Dense(units = 1))

#Compiling the RNN

In [ ]:
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

# Fitting the RNN to the Training set

In [ ]:
regressor.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/200
 193/3987 [>.............................] - ETA: 14:03 - loss: 0.0058

# Getting the real stock price of 22/7/2020

In [ ]:
dataset_test = pd.read_csv('Test.csv')
real_open_price = dataset_test.iloc[:, 1].values

# Getting the predicted stock prices of 22/7/2020

In [ ]:
dataset_total = pd.concat((dataset_train['Close', 'Open', 'High', 'Low', 'Volume'], dataset_test['Close', 'Open', 'High', 'Low', 'Volume']), axis = 0)
inputs = dataset_total[len(dataset_total)-len(dataset_test)-1170:].values
inputs = inputs.reshape(-1,1)
inputs = sc.transform(inputs)
X_test = []
for i in range(1170, 1560):
  X_test.append(inputs[i-1170:i,:])
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 5))
predicted_open_prices = regressor.predict(X_test)
predicted_open_prices = sc.inverse_transform(predicted_open_prices)

# Creating a data structure with 870 timesteps and 1 output

In [ ]:
X_train1 = []
y_train1 = []
for i in range(1170, len(training_set)):
  X_train1.append(training_set_scaled[i-1170:i,:])
  y_train1.append(training_set_scaled[i, 3])
X_train1, y_train1 = np.array(X_train1), np.array(y_train1)

# Reshaping

In [ ]:
X_train1 = np.reshape(X_train1, (X_train1.shape[0], X_train1.shape[1], 5))

# Initializing the RNN

In [ ]:
regressor1 = Sequential()

#Adding the first LSTM layer and some Dropout regularisation

In [ ]:
regressor1.add(LSTM(units = 128, return_sequences = True, input_shape = (X_train1.shape[1], 5)))
regressor1.add(Dropout(0.2))

#Adding the second LSTM layer and some Dropout regularisation

In [ ]:
regressor1.add(LSTM(units = 128, return_sequences = True))
regressor1.add(Dropout(0.2))

# Adding the third LSTM layer and some Dropout regularisation

In [ ]:
regressor1.add(LSTM(units = 128, return_sequences = True))
regressor1.add(Dropout(0.2))

#Adding the fourth LSTM layer and some Dropout regularisation

In [ ]:
regressor.add(LSTM(units = 128, return_sequences = True))
regressor.add(Dropout(0.2))

#Adding the fifth LSTM layer and some Dropout regularisation

In [ ]:
regressor1.add(LSTM(units = 128))
regressor1.add(Dropout(0.2))

#The output layer

In [ ]:
regressor1.add(Dense(units = 1))

#Compiling the RNN

In [ ]:
regressor1.compile(optimizer = 'adam', loss = 'mean_squared_error')

# Fitting the RNN to the Training set

In [ ]:
regressor1.fit(X_train1, y_train1, batch_size = 32, epochs = 200)

# Getting the real stock price of 22/7/2020

In [ ]:
dataset_test = pd.read_csv('Test.csv')
real_close_price = dataset_test.iloc[:, 3].values

# Getting the predicted stock prices of 22/7/2020

In [ ]:
X_test1 = []
for i in range(1170, 1560):
  X_test1.append(inputs[i-1170:i,:])
X_test1 = np.array(X_test1)
X_test1 = np.reshape(X_test1, (X_test1.shape[0], X_test1.shape[1], 5))
predicted_close_prices = regressor.predict(X_test1)
predicted_close_prices = sc.inverse_transform(predicted_close_prices)

In [ ]:
y_pred_change = np.subtract(predicted_close_prices, predicted_open_prices)
y_real_change = np.subtract(real_close_price, real_close_price)

In [ ]:
np.concatenate((y_pred_change.reshape(len(y_pred_change),1),y_real_change.reshape(len(y_real_change),1)),1)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(np.sign(y_pred_change),np.sign(y_real_change)) 